# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import numpy as np
import re 
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [2]:
sentiment = pd.read_csv('sentiment.csv')

In [3]:
ds = sentiment.sample(1000)

In [4]:
ds.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
ds.head()

,target,id,date,flag,user,text
601161,0,2221082319,Thu Jun 18 04:39:39 PDT 2009,NO_QUERY,justinadkins,I wish twitter worked on foreign phones
399156,0,2057078255,Sat Jun 06 12:19:50 PDT 2009,NO_QUERY,Spazzalyn,Who wants to help me get Oingo Biongo back tog...
423857,0,2062962323,Sun Jun 07 00:53:30 PDT 2009,NO_QUERY,thatreallybites,Pogo Stick Fail [Gif]: He tried to do a backfl...
263906,0,1986692343,Sun May 31 19:11:09 PDT 2009,NO_QUERY,sms40,I am finishing packing for camp and it is a NI...
70571,0,1693696135,Sun May 03 23:26:29 PDT 2009,NO_QUERY,Lannybanana,doing history internal


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# your code here
def clean_up(s):
    words = s.split()
    words = [w.lower() for w in words if not w.startswith('http://')]
    words = re.findall('[^\d\W]+', ' '.join(words))
    return ' '.join(words)

def tokenize(s):
    return word_tokenize(s)

def stem_and_lemmatize(l):
    l = ' '.join(l)
    stem = PorterStemmer().stem(l)
    lemm = WordNetLemmatizer().lemmatize(stem)
    return lemm

def remove_stopwords(l):
    stop_words = stopwords.words('english')
    return [i for i in l.split() if i not in stop_words]

In [6]:
ds['text_processed'] = ds['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize)\
                       .apply(remove_stopwords)

In [35]:
ds['text'].head()

601161             I wish twitter worked on foreign phones 
399156    Who wants to help me get Oingo Biongo back tog...
423857    Pogo Stick Fail [Gif]: He tried to do a backfl...
263906    I am finishing packing for camp and it is a NI...
70571                               doing history internal 
Name: text, dtype: object

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [8]:
# your code here
allwords = [w for words in ds['text_processed'] for w in words if len(w) > 1]

In [9]:
bow = {k:allwords.count(k) for k in allwords}
bow

{'wish': 13,
 'twitter': 14,
 'worked': 1,
 'foreign': 2,
 'phon': 2,
 'wants': 3,
 'help': 18,
 'get': 47,
 'oingo': 1,
 'biongo': 1,
 'back': 31,
 'together': 4,
 'think': 21,
 'done': 17,
 'oh': 30,
 'wasnt': 2,
 'born': 1,
 'decade': 1,
 'earlier': 2,
 'witness': 1,
 'pogo': 1,
 'stick': 1,
 'fail': 3,
 'gif': 1,
 'tried': 2,
 'backflip': 1,
 'finishing': 1,
 'packing': 2,
 'camp': 3,
 'nightmare': 1,
 'ggrrrrr': 1,
 'wake': 1,
 'tomorrow': 33,
 'morning': 12,
 'one': 29,
 'cranky': 1,
 'history': 2,
 'intern': 1,
 'candycenoel': 1,
 'baseball': 1,
 'today': 39,
 'thats': 13,
 'exciting': 3,
 'anyways': 2,
 'packed': 2,
 'realised': 3,
 'macbook': 2,
 'away': 8,
 'case': 2,
 'already': 6,
 'ful': 1,
 'buongiornodaisy': 1,
 'season': 2,
 'going': 45,
 'emotional': 1,
 'benjimmin': 1,
 'highest': 1,
 'placed': 1,
 'clubs': 1,
 'last': 18,
 'snort': 1,
 'people': 11,
 'feel': 25,
 'sorry': 15,
 'well': 24,
 'shit': 1,
 'helenfegan': 1,
 'god': 7,
 'imagine': 1,
 'us': 7,
 'pub': 1,
 '

In [10]:
# your code here
sorted_bag = sorted(bow.items(), key=lambda kv: kv[1], reverse=True)
sb = {k:v for k,v in sorted_bag[:5000]}
sb

{'good': 62,
 'get': 47,
 'going': 45,
 'quot': 45,
 'got': 44,
 'like': 43,
 'day': 43,
 'go': 41,
 'amp': 41,
 'work': 40,
 'today': 39,
 'im': 38,
 'lol': 36,
 'time': 35,
 'thanks': 34,
 'really': 34,
 'tomorrow': 33,
 'know': 32,
 'see': 32,
 'back': 31,
 'home': 31,
 'oh': 30,
 'one': 29,
 'much': 29,
 'still': 28,
 'new': 28,
 'love': 27,
 'want': 27,
 'feel': 25,
 'well': 24,
 'haha': 24,
 'night': 24,
 'sleep': 23,
 'thank': 22,
 'na': 22,
 'think': 21,
 'great': 20,
 'fun': 20,
 'getting': 20,
 'miss': 19,
 'bad': 19,
 'help': 18,
 'last': 18,
 'sad': 18,
 'yeah': 18,
 'need': 18,
 'done': 17,
 'watch': 17,
 'nice': 16,
 'right': 16,
 'watching': 16,
 'sorry': 15,
 'twitter': 14,
 'cant': 14,
 'tonight': 14,
 'big': 14,
 'would': 14,
 'wish': 13,
 'thats': 13,
 'sick': 13,
 'lt': 13,
 'hope': 13,
 'dont': 13,
 'next': 13,
 'wait': 13,
 'make': 13,
 'morning': 12,
 'may': 12,
 'thought': 12,
 'little': 12,
 'ready': 12,
 'yes': 12,
 'cool': 12,
 'follow': 12,
 'hey': 12,
 'bed

In [14]:
words = pd.DataFrame(sb, index=['values'])

In [12]:
words

,good,get,going,quot,got,like,day,go,amp,work,...,typ,indiespotting,shows,unfollowing,pysco,locked,slapping,ow,haley,gemabear
values,62,47,45,45,44,43,43,41,41,40,...,1,1,1,1,1,1,1,1,1,1


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [17]:
def find_features(document):
    word = set(document)
    return {w:(w in word) for w in words.columns}

In [ ]:
documents = list(zip(ds['text_processed'], np.where(ds['target'] == 4, True, False)))

[(['wish', 'twitter', 'worked', 'foreign', 'phon'], False),
 (['wants',
   'help',
   'get',
   'oingo',
   'biongo',
   'back',
   'together',
   'think',
   'done',
   'oh',
   'wasnt',
   'born',
   'decade',
   'earlier',
   'witness'],
  False),
 (['pogo', 'stick', 'fail', 'gif', 'tried', 'backflip'], False),
 (['finishing',
   'packing',
   'camp',
   'nightmare',
   'ggrrrrr',
   'wake',
   'tomorrow',
   'morning',
   'one',
   'cranky',
   'camp'],
  False),
 (['history', 'intern'], False),
 (['candycenoel', 'baseball', 'today', 'thats', 'exciting', 'anyways'], False),
 (['packed',
   'realised',
   'packed',
   'macbook',
   'away',
   'case',
   'already',
   'ful'],
  False),
 (['buongiornodaisy', 'season', 'going', 'emotional'], False),
 (['benjimmin',
   'highest',
   'placed',
   'clubs',
   'last',
   'season',
   'snort',
   'people',
   'feel',
   'sorry',
   'well',
   'shit'],
  True),
 (['helenfegan',
   'god',
   'imagine',
   'us',
   'together',
   'pub',
   'ma

In [18]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [19]:
# your code here
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

In [21]:
featuresets[0]

({'good': False,
  'get': False,
  'going': False,
  'quot': False,
  'got': False,
  'like': False,
  'day': False,
  'go': False,
  'amp': False,
  'work': False,
  'today': False,
  'im': False,
  'lol': False,
  'time': False,
  'thanks': False,
  'really': False,
  'tomorrow': False,
  'know': False,
  'see': False,
  'back': False,
  'home': False,
  'oh': False,
  'one': False,
  'much': False,
  'still': False,
  'new': False,
  'love': False,
  'want': False,
  'feel': False,
  'well': False,
  'haha': False,
  'night': False,
  'sleep': False,
  'thank': False,
  'na': False,
  'think': False,
  'great': False,
  'fun': False,
  'getting': False,
  'miss': False,
  'bad': False,
  'help': False,
  'last': False,
  'sad': False,
  'yeah': False,
  'need': False,
  'done': False,
  'watch': False,
  'nice': False,
  'right': False,
  'watching': False,
  'sorry': False,
  'twitter': True,
  'cant': False,
  'tonight': False,
  'big': False,
  'would': False,
  'wish': True,
  '

In [32]:
train_set, test_set = featuresets[200:], featuresets[:200]

In [33]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

0.67

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here
nltk.classify.accuracy(classifier, test_set)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here